In [13]:
import pandas as pd
from pathlib import Path 
from datetime import datetime
import sqlite3

In [14]:
conn = sqlite3.connect('LA_crime.sqlite')

In [15]:
cleaned_data = Path('Resources/Crime_Data_from_2020_to_Present.csv')
cleaned_data_df = pd.read_csv(cleaned_data)
cleaned_data_df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277


In [16]:
cleaned_data_df.drop(columns=['Date Rptd', 'TIME OCC','AREA', 'AREA NAME', 
                              'Rpt Dist No', 'Part 1-2', 'Crm Cd',
                              'Mocodes', 'Vict Age','Status','Vict Sex','Vict Age','Premis Cd','Premis Desc','Weapon Used Cd','Weapon Used Cd','Weapon Desc','Status','Status Desc', 'Crm Cd 1', 
                              'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 
                              'Cross Street'], inplace=True)

In [17]:
cleaned_data_df

,DR_NO,DATE OCC,Crm Cd Desc,Vict Descent,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,VEHICLE - STOLEN,O,34.0375,-118.3506
1,200106753,02/08/2020 12:00:00 AM,BURGLARY FROM VEHICLE,O,34.0444,-118.2628
2,200320258,11/04/2020 12:00:00 AM,BIKE - STOLEN,X,34.0210,-118.3002
3,200907217,03/10/2020 12:00:00 AM,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),O,34.1576,-118.4387
4,220614831,08/17/2020 12:00:00 AM,THEFT OF IDENTITY,H,34.0944,-118.3277
...,...,...,...,...,...,...
944230,242106032,02/17/2024 12:00:00 AM,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),O,34.2260,-118.6214
944231,242004546,01/16/2024 12:00:00 AM,BATTERY - SIMPLE ASSAULT,O,34.0690,-118.3054
944232,240104953,01/15/2024 12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),X,34.0685,-118.2460
944233,240309674,04/24/2024 12:00:00 AM,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",W,34.0215,-118.2868


In [18]:
cleaned_data_df[['Date', 'Time']] = cleaned_data_df['DATE OCC'].str.split(' ', n=1, expand=True)
cleaned_data_df.head()

,DR_NO,DATE OCC,Crm Cd Desc,Vict Descent,LAT,LON,Date,Time
0,190326475,03/01/2020 12:00:00 AM,VEHICLE - STOLEN,O,34.0375,-118.3506,03/01/2020,12:00:00 AM
1,200106753,02/08/2020 12:00:00 AM,BURGLARY FROM VEHICLE,O,34.0444,-118.2628,02/08/2020,12:00:00 AM
2,200320258,11/04/2020 12:00:00 AM,BIKE - STOLEN,X,34.0210,-118.3002,11/04/2020,12:00:00 AM
3,200907217,03/10/2020 12:00:00 AM,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),O,34.1576,-118.4387,03/10/2020,12:00:00 AM
4,220614831,08/17/2020 12:00:00 AM,THEFT OF IDENTITY,H,34.0944,-118.3277,08/17/2020,12:00:00 AM


In [19]:

cleaned_data_df[['Date', 'Time']] = cleaned_data_df['DATE OCC'].str.split(' ', n=1, expand=True)

columns_to_keep = ['DR_NO','Date','Time','Crm Cd Desc','Vict Descent','LAT', 'LON']

cleaned_data_df[columns_to_keep]


,DR_NO,Date,Time,Crm Cd Desc,Vict Descent,LAT,LON
0,190326475,03/01/2020,12:00:00 AM,VEHICLE - STOLEN,O,34.0375,-118.3506
1,200106753,02/08/2020,12:00:00 AM,BURGLARY FROM VEHICLE,O,34.0444,-118.2628
2,200320258,11/04/2020,12:00:00 AM,BIKE - STOLEN,X,34.0210,-118.3002
3,200907217,03/10/2020,12:00:00 AM,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),O,34.1576,-118.4387
4,220614831,08/17/2020,12:00:00 AM,THEFT OF IDENTITY,H,34.0944,-118.3277
...,...,...,...,...,...,...,...
944230,242106032,02/17/2024,12:00:00 AM,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),O,34.2260,-118.6214
944231,242004546,01/16/2024,12:00:00 AM,BATTERY - SIMPLE ASSAULT,O,34.0690,-118.3054
944232,240104953,01/15/2024,12:00:00 AM,VANDALISM - MISDEAMEANOR ($399 OR UNDER),X,34.0685,-118.2460
944233,240309674,04/24/2024,12:00:00 AM,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",W,34.0215,-118.2868


In [20]:
cleaned_data_df.drop(columns=['Time'])

cleaned_data_df.head()


,DR_NO,DATE OCC,Crm Cd Desc,Vict Descent,LAT,LON,Date,Time
0,190326475,03/01/2020 12:00:00 AM,VEHICLE - STOLEN,O,34.0375,-118.3506,03/01/2020,12:00:00 AM
1,200106753,02/08/2020 12:00:00 AM,BURGLARY FROM VEHICLE,O,34.0444,-118.2628,02/08/2020,12:00:00 AM
2,200320258,11/04/2020 12:00:00 AM,BIKE - STOLEN,X,34.0210,-118.3002,11/04/2020,12:00:00 AM
3,200907217,03/10/2020 12:00:00 AM,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),O,34.1576,-118.4387,03/10/2020,12:00:00 AM
4,220614831,08/17/2020 12:00:00 AM,THEFT OF IDENTITY,H,34.0944,-118.3277,08/17/2020,12:00:00 AM


In [21]:
# Convert 'Date' to datetime using the correct format
cleaned_data_df['Date'] = pd.to_datetime(cleaned_data_df['Date'], format='%m/%d/%Y')

# Extract the year and create a new column 'Year'
cleaned_data_df['Year'] = cleaned_data_df['Date'].dt.year

# Drop the original 'Date' column
cleaned_data_df = cleaned_data_df.drop(columns=['Date'])

# Display the updated DataFrame
cleaned_data_df.head()




,DR_NO,DATE OCC,Crm Cd Desc,Vict Descent,LAT,LON,Time,Year
0,190326475,03/01/2020 12:00:00 AM,VEHICLE - STOLEN,O,34.0375,-118.3506,12:00:00 AM,2020
1,200106753,02/08/2020 12:00:00 AM,BURGLARY FROM VEHICLE,O,34.0444,-118.2628,12:00:00 AM,2020
2,200320258,11/04/2020 12:00:00 AM,BIKE - STOLEN,X,34.0210,-118.3002,12:00:00 AM,2020
3,200907217,03/10/2020 12:00:00 AM,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),O,34.1576,-118.4387,12:00:00 AM,2020
4,220614831,08/17/2020 12:00:00 AM,THEFT OF IDENTITY,H,34.0944,-118.3277,12:00:00 AM,2020


In [22]:

# mapping dictionary
descent_mapping = {
    'A': 'Other Asian',
    'B': 'Black',
    'C': 'Chinese',
    'D': 'Cambodian',
    'F': 'Filipino',
    'G': 'Guamanian',
    'H': 'Hispanic/Latin/Mexican',
    'I': 'American Indian/Alaskan Native',
    'J': 'Japanese',
    'K': 'Korean',
    'L': 'Laotian',
    'O': 'Other',
    'P': 'Pacific Islander',
    'S': 'Samoan',
    'U': 'Hawaiian',
    'V': 'Vietnamese',
    'W': 'White',
    'X': 'Unknown',
    'Z': 'Asian Indian'
}


columns_to_keep = ['DR_NO','Crm Cd Desc', 'Vict Descent','LAT', 'LON', 'Year']

df_filtered = cleaned_data_df[columns_to_keep]

df_filtered.loc[:, 'Vict Descent'] = df_filtered['Vict Descent'].map(descent_mapping)

df_filtered.head()



,DR_NO,Crm Cd Desc,Vict Descent,LAT,LON,Year
0,190326475,VEHICLE - STOLEN,Other,34.0375,-118.3506,2020
1,200106753,BURGLARY FROM VEHICLE,Other,34.0444,-118.2628,2020
2,200320258,BIKE - STOLEN,Unknown,34.0210,-118.3002,2020
3,200907217,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),Other,34.1576,-118.4387,2020
4,220614831,THEFT OF IDENTITY,Hispanic/Latin/Mexican,34.0944,-118.3277,2020


In [23]:
cleaned_data_df.to_sql('LA_crime_data', conn, if_exists='replace', index=False)

944235

In [24]:
conn.close()